In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cleaned-indian-recipes-dataset/Cleaned_Indian_Food_Dataset.csv


## Install Packages

In [2]:
!pip install -q spacy scikit-learn
!python -m spacy download en_core_web_sm

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 974, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno -3] Temporary failure in name resolution

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
             

## Import Libraries

In [5]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Load Dataset

In [6]:
df = pd.read_csv("/kaggle/input/cleaned-indian-recipes-dataset/Cleaned_Indian_Food_Dataset.csv")

## Showing first 10 row

In [7]:
df.head(10)

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12
5,Pudina Khara Pongal Recipe (Rice and Lentils C...,"1 cup Rice - soaked for 20 minutes,1 Green Chi...",30,South Indian Recipes,"To begin making Pudina Khara Pongal Recipe, wa...",https://www.archanaskitchen.com/pudina-khara-p...,"cashew nuts,salt,yellow moong dal (split),ging...",https://www.archanaskitchen.com/images/archana...,15
6,Udupi Style Ash Gourd Coconut Curry Recipe,"4 Dry Red Chillies,1/2 teaspoon Turmeric powde...",40,Udupi,To begin making the Udupi Style Ash Gourd Coco...,https://www.archanaskitchen.com/udupi-style-as...,"salt,tamarind water,coconut,mustard seeds,jagg...",https://www.archanaskitchen.com/images/archana...,14
7,Mexican Style Black Bean Burrito Recipe,"6 to 8 Spinach Leaves (Palak),1/4 cup Black be...",40,Mexican,"To begin making the Black Bean Burrito recipe,...",https://www.archanaskitchen.com/black-bean-bur...,"tomato,spring onion greens,hung curd (greek yo...",https://www.archanaskitchen.com/images/archana...,12
8,Spicy Crunchy Masala Idli Recipe,"Mint Leaves (Pudina) - to taste,1/2 teaspoon T...",30,South Indian Recipes,"To prepare Spicy Crunchy Masala Idli Recipe, H...",https://www.archanaskitchen.com/spicy-crunchy-...,"tomato,salt,coriander (dhania) leaves,idli,red...",https://www.archanaskitchen.com/images/archana...,12
9,Cauliflower Leaves Chutney (Recipe in Hindi),"3 cloves garlic, 1 big Spoon oil, 2 tablespoo...",25,South Indian Recipes,"To make cauliflower leaf chutney, first of all...",https://www.archanaskitchen.com/cauliflower-le...,"tomato,salt,turmeric powder,red chillies,spoon...",https://www.archanaskitchen.com/images/archana...,13


## Show last 5 row

In [8]:
df.tail()

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
5933,Saffron Paneer Peda Recipe,"2 teaspoons Rose water,1/2 teaspoon Ghee,2 cup...",20,Indian,To begin making the Saffron Paneer Peda recipe...,https://www.archanaskitchen.com/saffron-paneer...,"saffron strands generous,ghee,paneer,rose wate...",https://www.archanaskitchen.com/images/archana...,7
5934,Italian Arancini Rice Balls Recipe With Delici...,"4 sprig Basil leaves - finely chopped,2 cups W...",100,Italian Recipes,To begin making the Italian Arancini Rice Ball...,https://www.archanaskitchen.com/italian-aranci...,"salt,gorgonzola cheese cubes,wheat bread crumb...",https://www.archanaskitchen.com/images/archana...,14
5935,Quinoa Phirnee Recipe (Quinoa Milk Pudding),"3/4 cup Sugar,2 cup Milk - vegans can substitu...",35,Indian,"To begin making Quinoa Phirnee Recipe, place a...",https://www.archanaskitchen.com/quinoa-phirnee...,"ghee,milk,saffron strands,sugar,cardamom powde...",https://www.archanaskitchen.com/images/archana...,8
5936,Ullikadala Pulusu Recipe | Spring Onion Curry,"1 pinch Turmeric powder (Haldi),1/2 teaspoon M...",15,Andhra,To begin making Ullikadala Pulusu Recipe | Spr...,https://www.archanaskitchen.com/ullikadala-pul...,"salt,tamarind water,mustard seeds,asafoetida (...",https://www.archanaskitchen.com/images/archana...,13
5937,Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...,"Salt - as required,3 to 4 Cloves (Laung),Ghee ...",75,Kashmiri,To begin making the Kashmiri Kokur Yakhni reci...,https://www.archanaskitchen.com/kashmiri-kokur...,"black cardamom (badi elaichi),salt,mustard oil...",https://www.archanaskitchen.com/images/archana...,13


In [9]:
df.shape

(5938, 9)

In [10]:
df.columns

Index(['TranslatedRecipeName', 'TranslatedIngredients', 'TotalTimeInMins',
       'Cuisine', 'TranslatedInstructions', 'URL', 'Cleaned-Ingredients',
       'image-url', 'Ingredient-count'],
      dtype='object')

In [11]:
df.isnull().sum()

TranslatedRecipeName      0
TranslatedIngredients     0
TotalTimeInMins           0
Cuisine                   0
TranslatedInstructions    0
URL                       0
Cleaned-Ingredients       0
image-url                 0
Ingredient-count          0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5938 entries, 0 to 5937
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   TranslatedRecipeName    5938 non-null   object
 1   TranslatedIngredients   5938 non-null   object
 2   TotalTimeInMins         5938 non-null   int64 
 3   Cuisine                 5938 non-null   object
 4   TranslatedInstructions  5938 non-null   object
 5   URL                     5938 non-null   object
 6   Cleaned-Ingredients     5938 non-null   object
 7   image-url               5938 non-null   object
 8   Ingredient-count        5938 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 417.6+ KB


In [13]:
df['Cuisine'].unique()

array(['Indian', 'South Indian Recipes', 'Andhra', 'Udupi', 'Mexican',
       'Fusion', 'Continental', 'Bengali Recipes', 'Punjabi', 'Chettinad',
       'Tamil Nadu', 'Maharashtrian Recipes', 'North Indian Recipes',
       'Italian Recipes', 'Sindhi', 'Thai', 'Chinese', 'Kerala Recipes',
       'Gujarati Recipes\ufeff', 'Coorg', 'Rajasthani', 'Asian',
       'Middle Eastern', 'Coastal Karnataka', 'European', 'Kashmiri',
       'Karnataka', 'Lucknowi', 'Hyderabadi', 'Side Dish', 'Goan Recipes',
       'Arab', 'Assamese', 'Bihari', 'Malabar', 'Himachal', 'Awadhi',
       'Cantonese', 'North East India Recipes', 'Sichuan', 'Mughlai',
       'Japanese', 'Mangalorean', 'Vietnamese', 'British',
       'North Karnataka', 'Parsi Recipes', 'Greek', 'Nepalese',
       'Oriya Recipes', 'French', 'Indo Chinese', 'Konkan',
       'Mediterranean', 'Sri Lankan', 'Uttar Pradesh', 'Malvani',
       'Indonesian', 'African', 'Shandong', 'Korean', 'American',
       'Kongunadu', 'Caribbean', 'South Karnat

## Loads a Pre-trained spaCy language model 

In [14]:
nlp = spacy.load("en_core_web_sm")

## Preprocessing 

In [15]:
def preprocess(text):
    text = str(text).lower()
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

## Clean Missing Data

In [16]:
df = df.dropna(subset=["TranslatedIngredients", "TranslatedInstructions"])

## Preprocess Ingredients in the Dataset

In [17]:
df["clean_ingredients"] = df["TranslatedIngredients"].apply(preprocess)

## TF-IDF Vectorization of the Ingredients

In [18]:
vectorizer = TfidfVectorizer()
ingredient_vectors = vectorizer.fit_transform(df["clean_ingredients"])

## Top Recipe Matches

In [19]:
def get_recipes(user_ingredients, top_n=5):
    cleaned_input = preprocess(user_ingredients)
    user_vector = vectorizer.transform([cleaned_input])
    similarity_scores = cosine_similarity(user_vector, ingredient_vectors).flatten()
    top_matches = similarity_scores.argsort()[-top_n:][::-1]
    return df.iloc[top_matches][[
        "TranslatedRecipeName", 
        "TranslatedIngredients", 
        "TranslatedInstructions", 
        "Cuisine"
    ]]

## Get Input from User and Results

In [23]:
user_input = input("Enter available ingredients (comma separated): ")
results = get_recipes(user_input)
for i, row in results.iterrows():
    print("\n" + "-"*30)
    print(f"Recipe: {row['TranslatedRecipeName']}")
    print(f"Cuisine: {row['Cuisine']}")
    print(f"Ingredients: {row['TranslatedIngredients']}")
    print(f"Instructions: {row['TranslatedInstructions']}")

Enter available ingredients (comma separated):  chicken



------------------------------
Recipe: Chicken Malai Kabab Recipe
Cuisine: North Indian Recipes
Ingredients: 4 Chicken - boneless,1/2 teaspoon Turmeric powder (Haldi) - to rub on chicken,Salt - as needed,Red Chilli powder - as required,3 teaspoon Fresh cream,4 teaspoon Ginger Garlic Paste
Instructions: To begin making the Chicken Malai Kabab recipe, firstly cut the chicken into small pieces for making kebabs.
Wash it clean and rub salt & turmeric on the boneless chicken & keep it aside for 10 minutes.After 10 minutes add ginger garlic paste, red chilli powder and the required salt.
Marinate for an hour.After an hour add fresh cream.
Once it is done, add 3 pieces on small skewers and place on a preheated grill pan.These kababs can be cooked on a frying pan also.
There is no need to add oil to the kebab.
Cook on a slow flame.
Until well cooked.
Serve Chicken Malai Kabab as a side dish with Dhaniya Pudina Chutney during ramadan or for weeknight dinners.


------------------------------
R